In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklvq import GMLVQ


matplotlib.rc("xtick", labelsize="small")
matplotlib.rc("ytick", labelsize="small")
data = np.genfromtxt('fourcl.csv', delimiter=',')
labels = np.genfromtxt('fourcl_labels.csv', delimiter=',')
#feature_names = [name[:-5] for name in iris.feature_names]


# Generalized Matrix LVQ (GMLVQ)

Example of how to use GMLVQ `[1]`_ on the classic iris dataset.


In [ ]:
labelpd = pd.read_csv('fourcl_labels.csv',header=None).to_numpy()

In [ ]:
np.shape(labelpd)

In [ ]:
labelpd = labelpd.squeeze()

In [ ]:
np.shape(labelpd)

In [ ]:
np.shape(data)

In [ ]:
labels = labelpd

In [ ]:
type(labels)

## Fitting the Model
Scale the data and create a GLVQ object with, e.g., custom distance function, activation
function and solver. See the API reference under documentation for defaults and other
possible parameters.



In [ ]:
# Sklearn's standardscaler to perform z-transform
scaler = StandardScaler()

# Compute (fit) and apply (transform) z-transform
data = scaler.fit_transform(data)

# The creation of the model object used to fit the data to.
model = GMLVQ(
    distance_type="adaptive-squared-euclidean",
    activation_type="sigmoid",
    activation_params={"beta": 2},
    solver_type="sgd",
    solver_params={"max_runs": 20,"batch_size":1,"step_size": np.array([0.1, 0.05])},
    random_state=1428,
)

The next step is to fit the GMLVQ object to the data and use the predict method to make the
predictions. Note that this example only works on the training data and therefor does not say
anything about the generalizability of the fitted model.



In [ ]:
# Train the model using the scaled data and true labels
model.fit(data, labels)

# Predict the labels using the trained model
# predicting on the same trianing data?
predicted_labels = model.predict(data)
print(len(data))
# To get a sense of the training performance we could print the classification report.
print(classification_report(labels, predicted_labels))

In [ ]:
feature_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']#, 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','ab','ac','ad','ae','af','ag','ah','ai','aj','ak']


## Extracting the Relevance Matrix
In addition to the prototypes (see GLVQ example), GMLVQ learns a
matrix `lambda_` which can tell us something about which features are most relevant for the
classification.



In [ ]:
# The relevance matrix is available after fitting the model.
relevance_matrix = model.lambda_

# Plot the diagonal of the relevance matrix
fig, ax = plt.subplots()
fig.suptitle("Relevance Matrix Diagonal")
ax.bar(feature_names, np.diagonal(relevance_matrix))
ax.set_ylabel("Weight")
ax.grid(False)

Note that the relevance diagonal adds up to one. The most relevant features for
distinguishing between the classes present in  the iris dataset seem to be (in decreasing
order) the petal length, petal width, sepal length, and sepal width. Although not very
interesting for the iris dataset one could use this information to select only the top most
relevant features to be used for the classification and thus reducing the dimensionality of
the problem.



## Transforming the data
In addition to making predictions GMLVQ can be used to transform the data using the
eigenvectors of the relevance matrix.



In [ ]:
# Transform the data (scaled by square root of eigenvalues "scale = True")
# data is the remaining datapoints.
transformed_data = model.transform(data, scale=True)

x_d = transformed_data[:, 0]
y_d = transformed_data[:, 1]

# Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
# prototypes are the 4 dots
transformed_model = model.transform(model.prototypes_, scale=True)
print(len(model.prototypes_))
x_m = transformed_model[:, 0]
y_m = transformed_model[:, 1]
#prototypes = ['PD','AD','HC']
# Plot
fig, ax = plt.subplots()
fig.suptitle("Discriminative projection Iris data and GMLVQ prototypes")
colors = ["black", "red", "green","blue"]
for i, cls in enumerate(model.classes_):
    ii = cls == labels
    ax.scatter(
        x_d[ii],
        y_d[ii],
        c=colors[i],
        s=100,
        alpha=0.7,
        edgecolors="white",
        label=model.classes_[model.prototypes_labels_[i]],
    )
ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
ax.set_xlabel("First eigenvector")
ax.set_ylabel("Second eigenvector")
ax.legend()
ax.grid(True)
print(model.classes_)
#plt.savefig('destination_path1.eps', format='eps')
print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

In [ ]:
# Plot the eigenvalues of the eigenvectors of the relevance matrix.
fig, ax = plt.subplots()
fig.suptitle("Eigenvalues")
ax.bar(range(0, len(model.eigenvalues_)), model.eigenvalues_)
ax.set_ylabel("Weight")
ax.grid(False)

# Plot the first two eigenvectors of the relevance matrix, which  is called `omega_hat`.
fig, ax = plt.subplots()
fig.suptitle("First Eigenvector")
ax.bar(feature_names, model.omega_hat_[:, 0])
ax.set_ylabel("Weight")
ax.grid(False)

fig, ax = plt.subplots()
fig.suptitle("Second Eigenvector")
ax.bar(feature_names, model.omega_hat_[:, 1])
ax.set_ylabel("Weight")
ax.grid(False)

In [ ]:
labels

In [ ]:
#print(np.where((labels == 'A1') & (labels == 'A2')))# and np.where(labels == 'A2')        
labelssubclass = labels
labelssubclass = np.where(labels == 'A1','A',labelssubclass) 
labelssubclass = np.where(labels == 'A2','A',labelssubclass) 
labelssubclass = np.where(labels == 'B1','B',labelssubclass) 
labelssubclass = np.where(labels == 'B2','B',labelssubclass) 


In [ ]:
labelssubclass

In [ ]:
labelssubclass.shape

In [ ]:
labels1 = np.where(labels == 'A1') 
labels2 = np.where(labels == 'A2') 
labels3 = np.where(labels == 'B1') 
labels4 = np.where(labels == 'B2') 

In [ ]:
#labelssubclass[labels1] = 'A1'
#labelssubclass[labels2] = 'A2'
#labelssubclass[labels3] = 'B1'
#labelssubclass[labels4] = 'B2'

In [ ]:
labelssubclass[3999
              ]

In [ ]:
data.shape

In [ ]:
# Train the model using the scaled data and true labels
model.fit(data, labelssubclass)

# Predict the labels using the trained model
# predicting on the same trianing data?
predicted_labels = model.predict(data)
# To get a sense of the training performance we could print the classification report.
print(classification_report(labelssubclass, predicted_labels))

In [ ]:
# The relevance matrix is available after fitting the model.
relevance_matrix1 = model.lambda_

# Plot the diagonal of the relevance matrix
fig, ax = plt.subplots()
fig.suptitle("Relevance Matrix Diagonal")
ax.bar(feature_names, np.diagonal(relevance_matrix1))
ax.set_ylabel("Weight")
ax.grid(False)

In [ ]:

transformed_data = model.transform(data, scale=True)

x_d = transformed_data[:, 0]
y_d = transformed_data[:, 1]

transformed_model = model.transform(model.prototypes_, scale=True)
print(len(model.prototypes_))
x_m = transformed_model[:, 0]
y_m = transformed_model[:, 1]

fig, ax = plt.subplots()
fig.suptitle("Discriminative projection Iris data and GMLVQ prototypes")
colors = ["black", "red"]#, "green","blue"]
for i, cls in enumerate(model.classes_):
    ii = cls == labelssubclass
    ax.scatter(
        x_d[ii],
        y_d[ii],
        c=colors[i],
        s=100,
        alpha=0.7,
        edgecolors="white",
        label=model.classes_[model.prototypes_labels_[i]],
    )
ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
ax.set_xlabel("First eigenvector")
ax.set_ylabel("Second eigenvector")
ax.legend()
ax.grid(True)
print(model.classes_)
plt.savefig('destination_path23.eps', format='eps')
print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

In [ ]:
#print(np.where((labels == 'A1') & (labels == 'A2')))# and np.where(labels == 'A2')              

The transformed data and prototypes can be used to visualize the problem in a lower dimension,
which is also the space the model would compute the distance. The axis are the directions which
are the most discriminating directions (combinations of features). Hence, inspecting the
eigenvalues and eigenvectors (axis) themselves can be interesting.



In [ ]:
# Plot the eigenvalues of the eigenvectors of the relevance matrix.
fig, ax = plt.subplots()
fig.suptitle("Eigenvalues")
ax.bar(range(0, len(model.eigenvalues_)), model.eigenvalues_)
ax.set_ylabel("Weight")
ax.grid(False)

# Plot the first two eigenvectors of the relevance matrix, which  is called `omega_hat`.
fig, ax = plt.subplots()
fig.suptitle("First Eigenvector")
ax.bar(feature_names, model.omega_hat_[:, 0])
ax.set_ylabel("Weight")
ax.grid(False)

fig, ax = plt.subplots()
fig.suptitle("Second Eigenvector")
ax.bar(feature_names, model.omega_hat_[:, 1])
ax.set_ylabel("Weight")
ax.grid(False)

In [ ]:
# are the eigen values of the data with A and B as labels
np.sqrt(model.eigenvalues_)*model.omega_hat_

In [ ]:
model.eigenvalues_.shape

In [ ]:
# are the eigen values of the data with A and B as labels
model.omega_hat_[:,1]

In [ ]:
import math 

In [ ]:
om0 = math.sqrt(model.eigenvalues_[0])*model.omega_hat_[:,0]

In [ ]:
om0

In [ ]:
om1 = math.sqrt(model.eigenvalues_[1])*model.omega_hat_[:,1]

In [ ]:
temporaray_omega = [[om1]]

In [ ]:
temp_omega = np.array([om1,om2])

In [ ]:
temp_omega.shape

In [ ]:
fig, ax = plt.subplots()
fig.suptitle("First temp omega entry")
ax.bar(feature_names, temporaray_omega[0,:])
ax.set_ylabel("Weight")
ax.grid(False)

In the plots from the eigenvalues and eigenvector we see a similar effects as we could see from
just the diagonal of `lambda_`. The two leading (most relevant or discriminating) eigenvectors
mostly use the petal length and petal width in their calculation. The diagonal of the
relevance matrix can therefor be considered as a summary of the relevances of the features.



## References
_`[1]` Schneider, P., Biehl, M., & Hammer, B. (2009). "Adaptive Relevance Matrices in Learning
Vector Quantization" Neural Computation, 21(12), 3532–3561, 2009.

